# Simulating Bike-Transit Trips


In [ ]:
#import packages
import sys
from pathlib import Path
import os
from datetime import datetime, timedelta, date

#import custom modules
import find_candidate_stops
import process_gtfs
import raptor_mapping
import viz_and_metrics

#import transit-routing from the transit-routing sub module
sys.path.insert(0,str(Path.cwd() / 'transit-routing'))

## Transit Schedule
This block processes a zipped GTFS file using transit-routing's "GTFS_wrapper.py" function. Zipped GTFS data can be retrieved directly from transit agencies or from GTFS aggregators like [Transitland](https://www.transit.land). As an example, GTFS data representing MARTA service between [2022-12-17 and 2023-04-21](https://transitfeeds.com/p/marta/65/20221215) is provided.

In [ ]:
gtfs_wrapper_inputs = {
    "gtfs_name": 'marta', # gtfs file must be in the transit-routing directory with the name {gtfs_name}_gtfs.zip
    "service_date": 20230301, # analysis date in YYYYMMDD format (must be within the service range of the GTFS data)
    "modes": [1,3], # transit modes to include (1 = heavy rail/metro, 2 = light rail/trams, 3 = buses)
}
process_gtfs.process_gtfs(gtfs_wrapper_inputs)

## Settings Dictionary
This dictionary is used to specify filepaths for the GTFS data, network data, and specify raptor settings.

In [ ]:
Path.cwd()

In [ ]:
settings = {
    'gtfs_fp': Path.cwd() / 'transit-routing/GTFS/marta', #filepath for processed GTFS files
    'gtfs_zip': Path.cwd() / 'transit-routing/marta_gtfs.zip', #filepath for original zipped GTFS files
    'service_date': date(2023,3,1), #select day to get service, a weekday was used for this study
    'crs': 'epsg:2240', # the desired projected CRS to use
    'thresh': 2 * 5280, #set candidate stop threshold distance in units of CRS (using bike service area from tcqsm page 5-20)
    'tazs_fp': Path.home() / 'Documents/BikewaySimData/Data/ARC/Model_Traffic_Analysis_Zones_2020.geojson', # filepath of TAZs or origins/POIs (see below)
    'keyid': 'OBJECTID', #column with the unique taz/origin id in it
    
    #this for making the transfers.txt file need for raptor
    'transfer_time': 2, # allowed transfer time in minutes (DOES NOT INLCUDE WAIT TIME)
    'walk_spd': 2.5, # assumed walking speed for transfers in miles per hour
    
    #these are specific to RAPTOR
    'NETWORK_NAME': 'marta',
    'MAX_TRANSFER': 2, # number of transfers allowed = max_transfer - 1
    'WALKING_FROM_SOURCE': 0, # if true (1), person can walk to a different transit station from start station
    'CHANGE_TIME_SEC': 30, # time to enter/exit vehicle in seconds (NOT WAIT TIME)
    'PRINT_ITINERARY': 0, # if running individual raptor trips, this prints the outputs to console
    'OPTIMIZED': 0, # read transit-routing documentation, set to 0 for this project
    
    #times to test, datetime(YYYY,MM,DD,HH,MM,SS,MS)
    'first_time': datetime(2023, 3, 1, 8, 0, 0, 0), # original start time is 9am
    'end_time': datetime(2023, 3, 1, 10, 0, 0, 0), # original end time is 10am
    'timestep': timedelta(minutes=15), # time increments to test (original is 9am,9:20am,9:40am,10am)
    'timelimit': timedelta(hours=1), # set the max allowed total travel time in hours
    'output_fp': Path.home() / 'Documents/TransitSimData/Data' #path where you want things to output 
    }
           

## Pre-processing Steps (only run this block once,  unless changing settings)
- Prepare Study Area
- Find Candidate Stops: 
- Create RAPTOR files

In [ ]:
# create transfers.txt (needed for RAPTOR)
process_gtfs.create_transfers(settings)

# function that creates the study area and various base layers
find_candidate_stops.process_studyarea(settings)

## Use outputted study area polygon to create network data
Follow instructions in the [Downloading_OSM.ipynb](../osm_download/Downloading_OSM.ipynb), [Step_1 Network_Filtering_and_Processing.ipynb](../network/Step_1_Network_Filtering.ipynb), and [Step_2_Network_Reconciliation.ipynb](../network/Step_2_Network_Reconciliation.ipynb) using the study area polygon created in previous block.

The filepath to the study area will be: "settings['output_fp'] / 'base_layers.gpkg'" in the "study area" layer.

In [ ]:
#replace with filepath to network data
settings['network_fp'] = Path.home() / 'Documents/TransitSimData/networks/final_network.gpkg'
settings['links_layer'] = 'links' # name of links layer (default is links)
settings['nodes_layer'] = 'nodes', # name of nodes layer (default is nodes)
settings['impedance'] = 'length_ft', # specify which column of the links data should be used for shortest path routing weights

#writes your network files to the gpkg for referencing later
find_candidate_stops.write_network_to_base_layers(settings)

## Get candidate stops
Finds all transit stops that are within specificed threshold and are a unique route

In [ ]:
candidate_stops_by_taz, centroids = find_candidate_stops.candidate_stops(settings)

In [ ]:
candidate_stops_by_taz.loc[candidate_stops_by_taz['stop_id']=='8753']

In [ ]:
candidate_stops_by_taz['taz_snapdist']

In [ ]:
candidate_stops_by_taz['stops_snapdist']

## Select source TAZ/census block IDs
For this report, one-to-all-possible TAZs from the Atlanta Regional Commission's [Traffic Analysis Zones TAZs](https://opendata.atlantaregional.com/maps/5d5ae31b66a94464be1a0539936ab764/about) were used. If using a different shapefile/geopackage/geojson, ensure that there is a unuique ID column and that the geometry is unprojected.

NOTE: At this time, the input geometry must be a polygon, but the centroid of each polygon will determine distance from the road network. For POI data suchs as grocery stores, buffer the points to turn them into polygons.

In [ ]:
# replace with ['*'] to do all tazs (not recommened due to run time)
select_tazs = [553,1071,1377]

## Specify Mode Specific Routing Settings
If doing a combination of modes (bike then walk, walk then bike), provide a tuple for the 'thresh' and 'spd' values.

In [ ]:
bike_settings = {
    'thresh': 5280 * 2, # set access/egress thresh in units of CRS
    'max_thresh': 5280 * 2 * 2, # set the max biking/walking amount in units of CRS
    'spd': 8, # miles per hour
    'mode':'bike',
    'impedance':'dist', # column to use for impedance/weights
    'allow_wrongway':False,
    'allow_bus_to_bus':False,
    'overwrite_existing': True,
    'rail_start':False # only allow trips to start near rail stations
    }

walk_settings = {
    'thresh': 5280 * 0.625, # set 
    'max_thresh': 5280 * 0.625 * 2, 
    'spd': 2.5,
    'mode':'walk',
    'impedance':'dist',
    'allow_wrongway':True,
    'allow_bus_to_bus':False,
    'overwrite_existing': True,
    'rail_start':False
    }

#this one assumes bike will be parked at start (does not consider availability of parking)
bikewalk_settings = {
    'thresh': (5280 * 2, 5280 * 0.625),
    'max_thresh': 5280 * (2+0.625),
    'spd': (8,2.5),
    'mode':'bikewalk',
    'impedance':'dist',
    'allow_wrongway':(False,True),
    'allow_bus_to_bus':False,
    'overwrite_existing': True,
    'rail_start':False
    }

## Create RAPTOR trip files

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# create bike trip files
find_candidate_stops.raptor_preprocessing(settings,bike_settings,select_tazs)

In [ ]:
# create walk trip files
find_candidate_stops.raptor_preprocessing(settings,walk_settings,select_tazs)

In [ ]:
# create bikewalk trip files
find_candidate_stops.raptor_preprocessing(settings,bikewalk_settings,select_tazs)
# can add more or comment out as needed

## Specify RAPTOR Settings and Run

In [ ]:
help(run_raptor)

In [ ]:
#410 minutes
#transit-routing uses current working directory to make 
cwd = os.getcwd()
os.chdir(Path.home()/'Documents/GitHub/BikewaySimDev/transitsim/transit-routing')

In [ ]:
run_raptor(select_tazs,settings,bike_settings)

In [ ]:
#160 minutes??
run_raptor(select_tazs,settings,walk_settings)

In [ ]:
#430minutes
run_raptor(select_tazs,settings,bikewalk_settings)

In [ ]:
#change back
os.chdir(cwd)

## Mapping and Visualizing RAPTOR Outputs
Map out each trip, accessibility, travel time, and modes utilized.

In [ ]:
impedance = 'dist'
modes = ['bike','walk','bikewalk']

for mode in modes:
    #map_routes(settings,impedance,mode,select_tazs,timedelta(hours=1))
    viz_and_metrics(settings,impedance,mode,select_tazs,timedelta(hours=1))

# Compact View (LTS)

In [ ]:
stop

In [ ]:
settings = {
    #changed network from marta to martalatest
    
    #these are for the pre-processing steps
    'gtfs_fp': Path.home() / 'Documents/GitHub/transit-routing/GTFS/martalatest', #filepath for processed GTFS files
    'gtfs_zip': Path.home() / 'Documents/GitHub/transit-routing/martalatest_gtfs.zip', #filepath for original zipped GTFS files
    'network_fp': Path.home() / 'Documents/TransitSimData/networks/final_network.gpkg', #fp to geopackage with a links and nodes layer
    'links_layer': 'lowstress_links', # name of links layer
    'nodes_layer': 'lowstress_nodes', # name of nodes layer
    'impedance': 'dist', # specify which column of the links data should be used for shortest path routing
    'service_date': date(2023,3,1),#date(2022, 11, 24), #select day to get service, a weekday was used for this study
    'crs': 'epsg:2240', # the desired projected CRS to use
    'thresh': 2 * 5280, #set candidate stop threshold distance in units of CRS (using bike service area from tcqsm page 5-20)
    'tazs_fp': Path.home() / 'Documents/NewBikewaySimData/Data/ARC/Model_Traffic_Analysis_Zones_2020.geojson', # filepath of TAZs or origins/POIs
    'keyid': 'OBJECTID', #column with the unique taz/origin id in it
    
    #this for making the transfers.txt file need for raptor
    'transfer_time': 2, # allowed transfer time in minutes (DOES NOT INLCUDE WAIT TIME)
    'walk_spd': 2.5, # assumed walking speed for transfers in miles per hour
    
    #these are specific to RAPTOR
    'NETWORK_NAME': 'martalatest',#'marta',
    'MAX_TRANSFER': 2, # no more than 1 transfer
    'WALKING_FROM_SOURCE': 0, # if true (1), person can walk to a different transit station from start station
    'CHANGE_TIME_SEC': 30, # time to enter/exit vehicle (NOT WAIT TIME)
    'PRINT_ITINERARY': 0, # if running individual raptor trips, this prints the outputs
    'OPTIMIZED': 0, # read transit-routing documentation, set to 0 for this project
    
    #times to test, datetime(YYYY,MM,DD,HH,MM,SS,MS)
    'first_time': datetime(2023, 3, 1, 8, 0, 0, 0), # original start time is 9am
    'end_time': datetime(2023, 3, 1, 10, 0, 0, 0), # original end time is 10am
    'timestep': timedelta(minutes=15), # time increments to test (original is 9am,9:20am,9:40am,10am)
    'timelimit': timedelta(hours=1), # set the max allowed total travel time in hours
    'output_fp': Path.home() / 'Documents/TransitSimData/LTS' #path where you want things to output 
    }

#select_tazs = ['1071']#['288','553','411','1071']

bikelts_settings = {
    'thresh': (5280 * 2, 5280 * 0.625),
    'max_thresh': 5280 * (2+0.625),
    'spd': (8,2.5),
    'mode':'bikewalk',
    'impedance':'dist',
    'allow_wrongway':(False,True),
    'allow_bus_to_bus':False,
    'overwrite_existing': True,
    'rail_start':True
    }


In [ ]:

#create transfers.txt
create_transfers(settings)

# function that creates the study area and various base layers
process_studyarea(settings)

candidate_stops_by_taz, centroids = candidate_stops(settings)

raptor_preprocessing(settings,bikelts_settings,select_tazs)

#change cwd Fix this later
cwd = os.getcwd()
os.chdir(Path.home()/'Documents/GitHub/transit-routing')

run_raptor(select_tazs,settings,bikelts_settings)

#change back
os.chdir(cwd)


In [ ]:

impedance = 'dist'
modes = ['bikewalk']

for mode in modes:
    map_routes(settings,impedance,mode,select_tazs,timedelta(hours=1))
    viz_and_metrics(settings,impedance,mode,select_tazs,timedelta(hours=1))

# Using model OD data

In [ ]:
settings = {
    #changed network from marta to martalatest
    
    #these are for the pre-processing steps
    'gtfs_fp': Path.home() / 'Documents/GitHub/transit-routing/GTFS/martalatest', #filepath for processed GTFS files
    'gtfs_zip': Path.home() / 'Documents/GitHub/transit-routing/martalatest_gtfs.zip', #filepath for original zipped GTFS files
    'network_fp': Path.home() / 'Documents/TransitSimData/networks/final_network.gpkg', #fp to geopackage with a links and nodes layer
    'links_layer': 'links', # name of links layer
    'nodes_layer': 'nodes', # name of nodes layer
    'impedance': 'dist', # specify which column of the links data should be used for shortest path routing
    'service_date': date(2023,3,1),#date(2022, 11, 24), #select day to get service, a weekday was used for this study
    'crs': 'epsg:2240', # the desired projected CRS to use
    'thresh': 2.5 * 5280, #set candidate stop threshold distance in units of CRS (using bike service area from tcqsm page 5-20)
    'tazs_fp': Path.home() / 'Documents/NewBikewaySimData/Data/ARC/Model_Traffic_Analysis_Zones_2020.geojson', # filepath of TAZs or origins/POIs
    'keyid': 'OBJECTID', #column with the unique taz/origin id in it
    
    #this for making the transfers.txt file need for raptor
    'transfer_time': 2, # allowed transfer time in minutes (DOES NOT INLCUDE WAIT TIME)
    'walk_spd': 3, # assumed walking speed for transfers in miles per hour
    
    #these are specific to RAPTOR
    'NETWORK_NAME': 'martalatest',#'marta',
    'MAX_TRANSFER': 2, # no more than 1 transfer
    'WALKING_FROM_SOURCE': 0, # if true (1), person can walk to a different transit station from start station
    'CHANGE_TIME_SEC': 30, # time to enter/exit vehicle (NOT WAIT TIME)
    'PRINT_ITINERARY': 0, # if running individual raptor trips, this prints the outputs
    'OPTIMIZED': 0, # read transit-routing documentation, set to 0 for this project
    
    #times to test, datetime(YYYY,MM,DD,HH,MM,SS,MS)
    'first_time': datetime(2023, 3, 1, 4, 30, 0, 0), # original start time is 9am
    'end_time': datetime(2023, 3, 2, 00, 30, 0, 0), # original end time is 10am
    'timestep': timedelta(minutes=15), # time increments to test (original is 9am,9:20am,9:40am,10am)
    'timelimit': timedelta(hours=1), # set the max allowed total travel time in hours
    'output_fp': Path.home() / 'Documents/TransitSimData/ABM', #path where you want things to output 

    #restrict to only starting at rail station
    'rail_start' : True
    }

#bring in od data
ods = pd.read_csv(settings['output_fp']/'ods.csv')
ods['origin'] = ods['origin'].astype(str)
ods['destination'] = ods['destination'].astype(str)
#all tazs
select_tazs = ods['origin'].unique().tolist()#['1071']#['288','553','411','1071']

#check if trip is within 1 hr of the departure time (ABM specific)
ods['year'] = settings['service_date'].year
ods['month'] = settings['service_date'].month
ods['day'] = settings['service_date'].day
ods['adj_time'] = pd.to_datetime(ods[['year','month','day','hour','minute']])
ods['adjusted'] = ods['adj_time'] + pd.to_timedelta(ods['depart_time'])

# bike_settings = {
#     'thresh': 5280 * 2.5, # set access/egress thresh
#     'max_thresh': 5280 * 2.5 * 2, # set the max biking/walking amount
#     'spd': 8,
#     'mode':'bike',
#     'impedance':'dist',
#     'allow_wrongway':False,
#     'allow_bus_to_bus':False,
#     'overwrite_existing': False
#     }

walk_settings = {
    'thresh': 5280 * 0.5,
    'max_thresh': 5280 * 0.5 * 2, #set to twice
    'spd': 3,
    'mode':'walk',
    'impedance':'dist',
    'allow_wrongway':True,
    'allow_bus_to_bus':True,
    'overwrite_existing': True
    }

#this one assumes bike will be parked at start (removes bus stops from first mile that aren't next to rail)
bikewalk_settings = {
    'thresh': (5280 * 2.5, 5280 * 0.5),
    'max_thresh': 5280 * (3),
    'spd': (8,2.5),
    'mode':'bikewalk',
    'impedance':'dist',
    'allow_wrongway':(False,True),
    'allow_bus_to_bus':True,
    'overwrite_existing': True
    }


In [ ]:

#create transfers.txt
create_transfers(settings)

# function that creates the study area and various base layers
process_studyarea(settings)

candidate_stops_by_taz, centroids = candidate_stops(settings)

raptor_preprocessing(settings,bikewalk_settings,select_tazs,ods)
raptor_preprocessing(settings,walk_settings,select_tazs,ods)


In [ ]:

#change cwd Fix this later
cwd = os.getcwd()
os.chdir(Path.home()/'Documents/GitHub/transit-routing')

#run_raptor(select_tazs,settings,bikewalk_settings,ods)
run_raptor(select_tazs,settings,walk_settings,ods)

#change back
os.chdir(cwd)


In [ ]:

impedance = 'dist'
modes = ['walk','bikewalk']

for mode in modes:#
    map_routes(settings,impedance,mode,select_tazs)
    viz_and_metrics(settings,impedance,mode,select_tazs)
